In [1]:
import hopsworks
import joblib
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import os
import sys
from xgboost import XGBRegressor

# Add utils to path
sys.path.append('..')
from utils import reddit_scraper as utils

Loading CryptoBERT...


In [3]:
# Login to Hopsworks
project = hopsworks.login(api_key_value=os.getenv("HOPSWORKS_API_KEY"))
mr = project.get_model_registry()

# Get trained model
model = mr.get_model("crypto_price_model", version=1)
fv = model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = model.download()

2026-01-15 17:02:15,701 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-15 17:02:15,705 INFO: Initializing external client
2026-01-15 17:02:15,706 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-15 17:02:17,609 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279131
2026-01-15 17:02:23,546 INFO: Initializing for batch retrieval of feature vectors


Downloading: 0.000%|          | 0/426894 elapsed<00:00 remaining<?

In [4]:
model = XGBRegressor()

model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
model

XGBRegressor(base_score='5.266871E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'int', 'float', 'float',
                            'int', 'float', 'float', 'float', 'float', 'float',
                            'int', 'int', 'float', 'float'],
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [5]:
# Test les fonctions
import requests
crypto_df = utils.fetch_latest_crypto_features()
current_price = crypto_df['close'].values[0]
print(f"✅ Price: ${current_price:.2f}")


mean_sentiment, count = utils.fetch_latest_sentiment()
print(f"✅ Sentiment: {mean_sentiment:.3f} (from {count} posts)")

# Create X avec les BONS noms de colonnes (sans timestamp)
X = crypto_df[['high', 'low', 'open', 'volume',
               'close_7d_ma', 'close_30d_ma', 'ma_7_30_cross',
               'rsi_14', 'atr_14', 'bb_bandwidth',
               'volume_7d_ma', 'volume_ratio',
               'day_of_week', 'month']].copy()

# Ajouter sentiment avec les noms attendus par le modèle
X['reddit_aggregated_sentiment_backfill_mean_sentiment'] = mean_sentiment
X['reddit_aggregated_sentiment_backfill_count'] = count

# Predict
predicted_price = model.predict(X)[0]
change = ((predicted_price - current_price) / current_price) * 100

print(f"\n🎯 PREDICTION:")
print(f"   Current:   ${current_price:.2f}")
print(f"   Predicted: ${predicted_price:.2f}")
print(f"   Change:    {change:+.2f}%")

✅ Price: $142.72
📰 Analyzing 12 recent posts...
Using device: cuda
   Progress: 0/12 (0%)
✅ Sentiment: 0.444 (from 12 posts)

🎯 PREDICTION:
   Current:   $142.72
   Predicted: $149.50
   Change:    +4.74%
